In [2]:
####Packages####
import pprint
from BCBio.GFF import GFFExaminer
import pandas as pd
import numpy as np
import shutil
import os
import glob
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from pathlib import Path
import re
import gff3_parser

In [3]:
#Pull in the Genome Annotation File for the correct species & Parse
os.chdir('/home/jake/Downloads/Genomes/gff_files/')
anotes=os.listdir('/home/jake/Downloads/Genomes/gff_files/')
for f in anotes:
    gff=os.path.basename(f)
    name=gff.split('.')[0] #Removes.gff
    _ = gff3_parser.parse_gff3(gff,verbose = False,  parse_attributes=True)
    df=_[['Seqid', 'Type', 'Start', 'End', 'Strand','gene', 'gene_biotype']]
    df.index.name='Index'
    df=df.loc[(df['Type']=='gene')& (df['gene_biotype']=='protein_coding')]
    df=df.astype({'Start':int,'End':int})
    #Create Filtered Gene Dataframes
    all_df=df.loc[(df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
    neg_df=df.loc[(df['Strand']=='-')& (df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] #Negative HSP
    pos_df=df.loc[(df['Strand']=='+')& (df['Type']=='gene') & (df['gene'].str.contains(r'hsp', re.I))] #Positive HSP
    alen=len(all_df)
    nlen=len(neg_df)
    plen=len(pos_df)
    print(f"{name} Genome imported")
    print(f"There are {alen} total hsp genes found in genome")
    print(f"{nlen} hsp genes are found on the negative strand")
    print(f"{plen} hsp genes are found on the positive strand")
    
    

Anguilla_anguilla Genome imported
There are 22 total hsp genes found in genome
8 hsp genes are found on the negative strand
14 hsp genes are found on the positive strand


In [4]:
#Annotated Gene Strands DF's
nstrand=df.loc[(df['Strand']=='-')].sort_values(by="Start", ascending=True)#Negative Strand Genes
#master_nstrand=df.loc[(df['Strand']=='-')].sort_values(by="Start", ascending=True)
pstrand=df.loc[(df['Strand']=='+')].sort_values(by="Start", ascending=True)#Negative Strand Genes
#List of Columns to manipulate
clist=['shift_1', 'shift_2', 'shift_3', 'shift_4', 'shift_5', 'shift_6', 'shift_7', 'shift_8', 'shift_9', 'shift_10']
print(f"Negative Strands Created Sucessfully")
print(f"Positive Strands Created Sucessfully")

Negative Strands Created Sucessfully
Positive Strands Created Sucessfully


In [5]:
#Function  manipulate the dataframes
def seq_manipulate(df):
    df['shift_1']=df['Start']- df['End'].shift(+1)
    df['shift_2']=df['Start']- df['End'].shift(+2)
    df['shift_3']=df['Start']- df['End'].shift(+3)
    df['shift_4']=df['Start']- df['End'].shift(+4)
    df['shift_5']=df['Start']- df['End'].shift(+5)
    df['shift_6']=df['Start']- df['End'].shift(+6)
    df['shift_7']=df['Start']- df['End'].shift(+7)
    df['shift_8']=df['Start']- df['End'].shift(+8)
    df['shift_9']=df['Start']- df['End'].shift(+9)
    df['shift_10']=df['Start']- df['End'].shift(+10)
    df[clist]= df[clist].clip(lower=0)#Remove all of the negative values and convert them to 0
    df[clist]= df[clist].fillna(0) #Convert NA values to 0
    df[clist]= df[clist].replace(0, np.nan) #Convert all the 0 values to NA
    df['inter_genic']=df[clist].min(1)
    df['Stop']= df['Start'] + df['inter_genic']
    df['combined']=df.apply(lambda x:'%s_%s' % (x['Start'],x['End']),axis=1)
    df.update('"' + df['combined'].astype(str) + '"')
#Transfrom both strands of DFs

seq_manipulate(nstrand)
print(f"Negative Strand Manipulated Sucessfully")
seq_manipulate(pstrand)
print(f"Positive Strand Manipulated Sucessfully")
#Create Two Filtered Data Sets
n_=nstrand[['Seqid', 'Type', 'Strand','gene', 'gene_biotype', 'Start', 'combined']]
p_=pstrand[['Seqid', 'Type', 'Strand','gene', 'gene_biotype', 'Start', 'combined']]
n_hsp=n_.loc[(n_['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
p_hsp=p_.loc[(p_['gene'].str.contains(r'hsp', re.I))] # ALL HSP genes
hsps= pd.concat([n_hsp,p_hsp])
hsps=hsps.sort_values(by="Start", ascending=True)
hsps['Chromsome']= hsps['Seqid'].apply(lambda x: x[-4:-2])
print(f"Chromosome Column Created")

Negative Strand Manipulated Sucessfully
Positive Strand Manipulated Sucessfully
Chromosome Column Created


In [6]:
#####Need to read in all chromosome fasta files, and save them as variables chr##
genome=name.lower()
os.chdir(f"/home/jake/Downloads/Genomes/{genome}/")
filess=os.listdir(f"/home/jake/Downloads/Genomes/{genome}/")
print(filess)
for f in filess:
    name_=os.path.basename(f)
    #file_out=dest+'trim_'+name
    for f in SeqIO.parse(name_, "fasta"):
        if name_=='chr1':
            chr1=
        

SyntaxError: invalid syntax (3698096884.py, line 11)

In [ ]:
##Change directory into Emboss and run the script
os.chdir('/home/jake/Downloads/Bio_tools/EMBOSS/EMBOSS-6.6.0/emboss/')
genome=name.lower()
for f in range(len(hsps)):
    for f in hsps['gene']:
        gene=(f)
        for f in hsps['Chromsome']:
            chromosome=(f)
            genome_path= (f"/home/jake/Downloads/Genomes/{name}/chr{chromosome}.fna/")
            for f in hsps['combined']:
                theta=(f)
                os.system(f"./extractseq {genome_path} result_{genome}_{gene}.seq -regions {theta}")


NameError: name 'name' is not defined